In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.simplefilter("ignore") # Silence warnings

import xarray as xr
import numpy as np

from tqdm import tqdm
import bokeh

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as mticker
import matplotlib.patches as patches
import cartopy
import cartopy.crs as ccrs
from cartopy import util
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import cmocean
import pop_tools 

from pop_tools import get_grid
import proplot
from utils import *

In [2]:
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client

def CLSTR(N=1,T=30,M=109, P=10):
    nnodes = N; ntime="00:"+str(T)+":00"; nmemory= str(M)+"GB"; nprocesses= P
    cluster = PBSCluster(
        cores=nnodes, # The number of cores you want
        memory=nmemory, # Amount of memory
        processes=nprocesses, # How many processes
        walltime=ntime, # Amount of wall time
        queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
        local_directory='/glade/scratch/yeddebba/tmp', # Use your local directory
#         resource_spec='select=1:ncpus=1:mem=10GB', # Specify resources
        project='USIO0028', # Input your project ID here
        interface='ib0', # Interface to use
    )
    cluster.scale(nnodes*nprocesses)
    dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})
    client = Client(cluster)
    
    return client


In [19]:
C=CLSTR(1,15,209,10)
C

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/yeddebba/proxy/41260/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/yeddebba/proxy/41260/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.47:33369,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/yeddebba/proxy/41260/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [20]:
C

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/yeddebba/proxy/41260/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/yeddebba/proxy/41260/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.47:33369,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/yeddebba/proxy/41260/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [21]:
ds = xr.open_mfdataset('/glade/campaign/cesm/development/bgcwg/projects/hi-res_JRA/cases/g.e22.G1850ECO_JRA_HR.TL319_t13.004/output/ocn/proc/tseries/day_1/g.e22.G1850ECO_JRA_HR.TL319_t13.004.pop.h.nday1.SST.00340101-00341231.nc')
ds

In [28]:
dc = xr.open_mfdataset('/glade/campaign/cesm/development/bgcwg/projects/hi-res_JRA/cases/g.e22.G1850ECO_JRA_HR.TL319_t13.004/output/ocn/proc/tseries/day_1/g.e22.G1850ECO_JRA_HR.TL319_t13.004.pop.h.nday1.FG_CO2_2.00340101-00341231.nc')
dc

<xarray.Dataset>
Dimensions:                 (z_t: 62, z_t_150m: 15, z_w: 62, z_w_top: 62, z_w_bot: 62, nlat: 2400, nlon: 3600, time: 365, d2: 2)
Coordinates:
  * z_t                     (z_t) float32 500.0 1.5e+03 ... 5.625e+05 5.875e+05
  * z_t_150m                (z_t_150m) float32 500.0 1.5e+03 ... 1.45e+04
  * z_w                     (z_w) float32 0.0 1e+03 2e+03 ... 5.5e+05 5.75e+05
  * z_w_top                 (z_w_top) float32 0.0 1e+03 ... 5.5e+05 5.75e+05
  * z_w_bot                 (z_w_bot) float32 1e+03 2e+03 ... 5.75e+05 6e+05
    ULONG                   (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
    ULAT                    (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
    TLONG                   (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
    TLAT                    (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
  * time                    (time) object 0034-01-02 00:00:00 ... 0035-01-01 ...
Dimensions without coordinates: nlat, nlon, d2
Data variables: (12/52)
    dz                      (z_t) float32 dask.array<chunksize=(62,), meta=np.ndarray>
    dzw                     (z_w) float32 dask.array<chunksize=(62,), meta=np.ndarray>
    KMT                     (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
    KMU                     (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
    REGION_MASK             (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
    UAREA                   (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
    ...                      ...
    salinity_factor         float64 ...
    sflux_factor            float64 ...
    nsurface_t              float64 ...
    nsurface_u              float64 ...
    time_bound              (time, d2) object dask.array<chunksize=(365, 2), meta=np.ndarray>
    FG_CO2_2                (time, nlat, nlon) float32 dask.array<chunksize=(365, 2400, 3600), meta=np.ndarray>
Attributes:
    title:             g.e22.G1850ECO_JRA_HR.TL319_t13.004
    history:           none
    Conventions:       CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-cu...
    time_period_freq:  day_1
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    contents:          Diagnostic and Prognostic Variables
    source:            CCSM POP2, the CCSM Ocean Component
    revision:          $Id$
    calendar:          All years have exactly  365 days.
    start_time:        This dataset was created on 2021-01-20 at 05:21:24.9
    cell_methods:      cell_methods = time: mean ==> the variable values are ...

In [35]:
do = xr.open_mfdataset('/glade/campaign/cesm/development/bgcwg/projects/hi-res_JRA/cases/g.e22.G1850ECO_JRA_HR.TL319_t13.004/output/ocn/proc/tseries/day_1/g.e22.G1850ECO_JRA_HR.TL319_t13.004.pop.h.nday1.STF_O2_2.00340101-00341231.nc')
do

<xarray.Dataset>
Dimensions:                 (z_t: 62, z_t_150m: 15, z_w: 62, z_w_top: 62, z_w_bot: 62, nlat: 2400, nlon: 3600, time: 365, d2: 2)
Coordinates:
  * z_t                     (z_t) float32 500.0 1.5e+03 ... 5.625e+05 5.875e+05
  * z_t_150m                (z_t_150m) float32 500.0 1.5e+03 ... 1.45e+04
  * z_w                     (z_w) float32 0.0 1e+03 2e+03 ... 5.5e+05 5.75e+05
  * z_w_top                 (z_w_top) float32 0.0 1e+03 ... 5.5e+05 5.75e+05
  * z_w_bot                 (z_w_bot) float32 1e+03 2e+03 ... 5.75e+05 6e+05
    ULONG                   (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
    ULAT                    (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
    TLONG                   (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
    TLAT                    (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
  * time                    (time) object 0034-01-02 00:00:00 ... 0035-01-01 ...
Dimensions without coordinates: nlat, nlon, d2
Data variables: (12/52)
    dz                      (z_t) float32 dask.array<chunksize=(62,), meta=np.ndarray>
    dzw                     (z_w) float32 dask.array<chunksize=(62,), meta=np.ndarray>
    KMT                     (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
    KMU                     (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
    REGION_MASK             (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
    UAREA                   (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
    ...                      ...
    salinity_factor         float64 ...
    sflux_factor            float64 ...
    nsurface_t              float64 ...
    nsurface_u              float64 ...
    time_bound              (time, d2) object dask.array<chunksize=(365, 2), meta=np.ndarray>
    STF_O2_2                (time, nlat, nlon) float32 dask.array<chunksize=(365, 2400, 3600), meta=np.ndarray>
Attributes:
    title:             g.e22.G1850ECO_JRA_HR.TL319_t13.004
    history:           none
    Conventions:       CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-cu...
    time_period_freq:  day_1
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    contents:          Diagnostic and Prognostic Variables
    source:            CCSM POP2, the CCSM Ocean Component
    revision:          $Id$
    calendar:          All years have exactly  365 days.
    start_time:        This dataset was created on 2021-01-20 at 05:21:24.9
    cell_methods:      cell_methods = time: mean ==> the variable values are ...

In [ ]:
ds.SST.isel(time=20).plot(figsize=(10,4),vmin=5,vmax=30,cmap='cmo.thermal')
# plt.ylim(1300,1900)
# plt.xlim(0,1000)

dc.FG_CO2_2.isel(time=20).plot(figsize=(10,4),vmin=-0.02,vmax=0.02,cmap='cmo.balance')
# plt.ylim(1300,1900)
# plt.xlim(0,1000)

do.STF_O2_2.isel(time=20).plot(figsize=(10,4),vmin=-0.1,vmax=0.1,cmap='cmo.balance')
# plt.ylim(1300,1900)
# plt.xlim(0,1000)